# Отток клиентов

Из банка стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from matplotlib import pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('/datasets/Churn.csv')

In [3]:
df.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


  Признаки
  
- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — сколько лет человек является клиентом банка
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата


  Целевой признак
  
- Exited — факт ухода клиента

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [5]:
df['Exited'].unique()

array([1, 0])

In [6]:
df.drop(['Surname'], axis=1, inplace=True)

In [7]:
sum(df['Exited']/len(df)) 
#доля объектов положительного класса

0.20369999999999389

In [8]:
1-sum(df['Exited']/len(df))
#доля объектов отрицательного класса

0.7963000000000061

In [9]:
df['Exited'].value_counts(normalize = True)

0    0.7963
1    0.2037
Name: Exited, dtype: float64

Доля объектов положительного класса 0.2,а отрицательного 0.8. 

Объектов отрицательного класса больше в 4 раза

In [10]:
#One-Hot Encoding
gender_ohe = pd.get_dummies(df['Gender'], drop_first=True)
country_ohe= pd.get_dummies(df['Geography'], drop_first=True)
df_ohe = pd.concat([df, gender_ohe, country_ohe], axis=1)

In [11]:
df_ohe.head()

,RowNumber,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Male,Germany,Spain
0,1,15634602,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,2,15647311,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0,0,0,1
2,3,15619304,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,4,15701354,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,5,15737888,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0,0,0,1


In [12]:
df_ohe.drop(['Gender', 'Geography', 'RowNumber'], axis=1, inplace=True)

In [13]:
features = df_ohe.drop("Exited", axis=1)
target = df_ohe["Exited"]

In [14]:
features.fillna(1, inplace=True)

- Еще можно заполнить пропуски нулями и предположить, что это новые клиенты.    
- Распределение по `Tenure` ровномерное, если сейчас заменить почти 900 пропусков (9% данных) средним или медианой, то мы значительно исказим распределение. В таком случае можно заполнить пропуски случайными значениями в интервале от минимального до максимального значения по данному столбцу.

- Так как значений в этом признаке ограниченное количество, можно сделать его категориальным признаком. Пропуски можно считать как за отдельную категорию (заполнить значением -1). Затем заменить тип данных `Tenure` на `object` и применить технику [OHE]

## Исследование задачи

### Деление на обучающую, валидационную и тестовую выборки

In [15]:
features_train, features_test1, target_train, target_test1 = train_test_split(features, target, 
                                                                              test_size=0.4, 
                                                                              random_state=1,
                                                                              stratify = target)

In [16]:
features_valid, features_test, target_valid, target_test = train_test_split(features_test1, target_test1, 
                                                                              test_size=0.5, 
                                                                              random_state=1,
                                                                              stratify = target_test1)

features_train, target_train - обучающая выборка  
features_valid, target_valid - валидационная выборка  
features_test, target_test - тестовая выборка  

In [17]:
print(features_train.shape[0])
print(features_valid.shape[0])
print(features_test.shape[0])

6000
2000
2000


### Масштабирование признаков

In [18]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [19]:
scaler = StandardScaler()
scaler.fit(features_train[numeric])

StandardScaler()

In [20]:
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

### Обучение моделей

In [21]:
#LogisticRegression
model = LogisticRegression()
model.fit(features_train, target_train)
predict = model.predict(features_valid)
print('Качество', accuracy_score(target_valid, predict))
print('F1:', f1_score(predict, target_valid))
print('AUC-ROC:', roc_auc_score(target_valid, predict))

Качество 0.7965
F1: 0.0
AUC-ROC: 0.5


In [22]:
%%time
#RandomForestClassifier
best_accuracy = 0
best_depth = 0
for depth in range(1,15):
    for est in range(1,25):
        model = RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=12345)
        model.fit(features_train, target_train)
        predict = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predict)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_depth = depth
            best_est = est
            
print('Глубина дерева:', best_depth, 'Количество деревьев:', best_est, 'Качество:', best_accuracy)
print('F1:', f1_score(predict, target_valid))
print('AUC-ROC:', roc_auc_score(target_valid, predict))

Глубина дерева: 12 Количество деревьев: 20 Качество: 0.862
F1: 0.5612403100775194
AUC-ROC: 0.704467950230662
CPU times: user 19.8 s, sys: 29.2 ms, total: 19.8 s
Wall time: 20 s


In [23]:
#DecisionTreeClassifier
best_accuracy = 0
best_depth = 0
for depth in range(1,15):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predict)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth
print('Глубина дерева:', best_depth, 'Качество:', best_accuracy)
print('F1:', f1_score(predict, target_valid))
print('AUC-ROC:', roc_auc_score(target_valid, predict))

Глубина дерева: 6 Качество: 0.857
F1: 0.5148771021992238
AUC-ROC: 0.6920549208684802


Наилучшие показатели у модели RandomForestClassifier с параметрами max_depth=12, n_estimators=20

## Борьба с дисбалансом

### Upsample

In [24]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [25]:
features_upsampled_train, target_upsampled_train = upsample(features_train, target_train, repeat=4)

In [26]:
print(target_upsampled_train.value_counts(normalize = 1))

1    0.50569
0    0.49431
Name: Exited, dtype: float64


In [27]:
best_accuracy = 0
best_depth = 0
for depth in range(1,11):
    for est in range(1, 11):
        model = RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=12345)
        model.fit(features_upsampled_train, target_upsampled_train)
        predict = model.predict(features_valid)
        accuracy = model.score(features_valid, target_valid)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_depth = depth
            best_est = est
            
print('Глубина дерева:', best_depth, 'Количество деревьев:', best_est, 'Качество:', best_accuracy)
print('F1:', f1_score(predict, target_valid))
print('AUC-ROC:', roc_auc_score(target_valid, predict))

Глубина дерева: 10 Количество деревьев: 7 Качество: 0.809
F1: 0.5770065075921907
AUC-ROC: 0.7486269011692741


### Downsample

In [28]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [29]:
features_downsampled_train, target_downsampled_train = downsample(features_train, target_train, fraction=0.5)

In [30]:
print(target_downsampled_train.value_counts(normalize = 1))

0    0.66159
1    0.33841
Name: Exited, dtype: float64


In [31]:
best_accuracy = 0
best_depth = 0
for depth in range(1,11):
    for est in range(1, 11):
        model = RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=12345)
        model.fit(features_downsampled_train, target_downsampled_train)
        predict = model.predict(features_valid)
        accuracy = model.score(features_valid, target_valid)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_depth = depth
            best_est = est
            
print('Глубина дерева:', best_depth, 'Количество деревьев:', best_est, 'Качество:', best_accuracy)
print('F1:', f1_score(predict, target_valid))
print('AUC-ROC:', roc_auc_score(target_valid, predict))

Глубина дерева: 3 Количество деревьев: 8 Качество: 0.8525
F1: 0.6038709677419355
AUC-ROC: 0.7454102793085844


In [32]:
#Создаем константную модель
target_predict_constant = pd.Series([0]*len(target_valid))
target_predict_constant.value_counts()
print('accuracy_score константой модели:', accuracy_score(target_valid, target_predict_constant))
print('AUC-ROC константой модели:', roc_auc_score(target_valid, target_predict_constant))

accuracy_score константой модели: 0.7965
AUC-ROC константой модели: 0.5


В первоначальные данных наблюдался значительный дисбаланс (80% ответов целевого признака были негативными и только 20% позитивными)

Устранен дисбаланс классов в обучающей выборки. Количество значений позитивного класса увеличено в 4 раза. Достигнут баланс классов обучеющей выборки: 1 = 0.50569, 0 = 0.49431.

На новых данных модель RandomForest показала результат выше, чем на несбалансированной выборке:  
F1: 0.592274678111588  
AUC-ROC: 0.7609119134542863


Финальная модель прошла проверку на адекватность в сравнении с контантной моделью:   
accuracy_score константой модели: 0.796  
accuracy_score финальной модели: 0.813  
AUC-ROC константой модели: 0.5  
AUC-ROC финальной модели: 0.761  

## Тестирование модели

In [33]:
model = RandomForestClassifier(max_depth=3, 
                               n_estimators=8, 
                               random_state=12345)
model.fit(features_downsampled_train, target_downsampled_train)
accuracy = model.score(features_test, target_test)
predict = model.predict(features_test)
print('Качество:', accuracy)
print('F1:', f1_score(target_test, predict))
print('AUC-ROC:', roc_auc_score(target_test, predict))
print('Точность: ', precision_score(target_test, predict))
print('Полнота:', recall_score(target_test, predict))

Качество: 0.8515
F1: 0.5308056872037914
AUC-ROC: 0.6879803428909251
Точность:  0.7466666666666667
Полнота: 0.4117647058823529


# Вывод

Финальная модель достигла метрики F1 = 0.59  и показывает адекватные результаты.

Удалось достигнуть показателя точности - 0.63. Модель верно предсказывает 63% ухода клиентов.

С точки зрения бизнеса полученная модель поможет маркетологам лучше определять килентов, которые могут уйти в ближайшее время.